# Optimization \[20 points\]


<b><span style="color:#00CED1">Association:</span></b>
<span style="color:#00CED1">Otto-Friedrich University of Bamberg</span>
<span style="color:#00CED1">Chair of Explainable Machine Learning (xAI)</span>
<span style="color:#00CED1">Deep Learning Assignments</span>

<b><span style="color:#00CED1">Description:</span></b>
<span style="color:#00CED1">This notebook introduces different optimization methods including (Stochastic) Gradient Descent, Momentum, RMSProp and Adam.</span>
<span style="color:#00CED1">Students will implement all of those and learn how each of them lead to different results.</span>

<span style="color:#00CED1"><b>Author:</b> Sebastian Doerrich</span>
<span style="color:#00CED1"><b>Copyright:</b> Copyright (c) 2022, Chair of Explainable Machine Learning (xAI), Otto-Friedrich University of Bamberg</span>
<span style="color:#00CED1"><b>Credits:</b> Christian Ledig, Sebastian Doerrich</span>
<span style="color:#00CED1"><b>License:</b> CC BY-SA</span>
<span style="color:#00CED1"><b>Version:</b> 1.0</span>
<span style="color:#00CED1"><b>Python:</b> Python 3</span>
<span style="color:#00CED1"><b>Maintainer:</b> Sebastian Doerrich</span>
<span style="color:#00CED1"><b>Email:</b> sebastian.doerrich@uni-bamberg.de</span>
<span style="color:#00CED1"><b>Status:</b> Production</span>

## Context
Welcome to the next part of your third programming assignment.
In this part you will implement different optimization methods and learn how all of them lead to different results.

## Motivation
Optimization of your model's parameters during training is necessary, to ensure convergence and good prediction results. Until now, you've always used Gradient Descent to update the parameters and minimize the cost. In this notebook, you'll gain skills with some more advanced optimization methods that can speed up learning and perhaps even get you to a better final value for the cost function. Having a good optimization algorithm can be the difference between waiting days vs. just a few hours to get a good result.

By the end of this notebook, you'll be able to:

* Apply optimization methods such as (Stochastic) Gradient Descent, Momentum, RMSProp and Adam
* Use random minibatches to accelerate convergence and improve optimization

Let's get started!

## Instructions
- You will be using Python 3.
- After coding your function, run the cell right below it to check if your result is correct.

## Important Notes for Your Submission
Before submitting your assignment, please make sure you are not doing the following:

1. You have not added any _extra_ `print` statement(s) in the assignment.
2. You have not added any _extra_ code cell(s) in the assignment.
3. You have not changed any of the function parameters.
4. You are not using any global variables inside your graded exercises. Unless specifically instructed to do so, please refrain from it and use the local variables instead.
5. You are not changing the assignment code where it is not required, like creating _extra_ variables.

If you do any of the mentioned, our test scripts will fail and as a result you will receive **0 points** for the respective task.

## Table of Contents
- [0 - Import the Necessary Libraries](#0)
- [1 - Visualization](#1)
- [2 - Gradient Descent](#2)
    - [2.1 - Gradient Descent Update Rule](#2-1)
    - [2.2 - Stochastic Gradient Descent (SGD)](#2-2)
    - [2.3 - Batch Gradient Descent](#2-3)
    - [2.4 - Mini-Batch Gradient Descent](#2-4)
- [3 - Momentum](#3)
    - [3.1 - Initialize the Velocity](#3-1)
    - [3.2 - Update the Model’s Parameters with Momentum](#3-2)
- [4 - Adam](#4)
    - [4.1 - Initialize Adam](#4-1)
    - [4.2 - Update Parameters using Adam](#4-2)
- [5 - Model with different Optimization algorithms](#5)
    - [5.1 - Mini-Batch Gradient Descent](#5-1)
    - [5.2 - Mini-Batch Gradient Descent with Momentum](#5-2)
    - [5.3 - Mini-Batch with Adam](#5-3)
    - [5.4 - Summary](#5-4)
- [6 - Learning Rate Decay and Scheduling](#6)
    - [6.1 - Decay on Every Iteration](#6-1)
    - [6.2 - Fixed Interval Scheduling](#6-2)
    - [6.3 - Using Scheduled Learning Rate Decay for each Optimization Method](#6-3)
        - [6.3.1 - Gradient Descent with Scheduled Learning Rate Decay](#6-3-1)
        - [6.3.2 - Gradient Descent with Momentum and Scheduled Learning Rate Decay](#6-3-2)
        - [6.3.3 - Adam with Scheduled Learning Rate Decay](#6-3-3)
    - [6.4 - Achieving similar performance with different methods](#6-4)
- [7 - End of Exercise](#7)

<a name='0'></a>
## 0 - Import the Necessary Libraries ##

In [ ]:
# Import packages
import numpy as np
import math
import sklearn
import sklearn.datasets
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

<a id='1'></a>
## 1 - Visualization ##

These functions will be used for visualization. We provide those for you, so you can plot your results. You don't have to code anything here.

In [ ]:
class Visualization:
    """
    This class is used to collect all visualizations.
    """

    @staticmethod
    def plot_the_cost(axes, iterations: list, costs: list, learning_rate: float):
        """
        Plot the cost of the network during training.

        :param iterations: Training iterations in the form [0, 1000, 2000, ..., 15000].
        :param costs: Costs of the network for each training iteration.
        :param learning_rate: Learning rate of the optimization algorithm.
        """

        axes.plot(iterations, costs, label='cost')
        axes.set_ylabel('Cost')
        axes.set_xlabel('Iterations')
        axes.set_title("Learning rate =" + str(learning_rate))
        axes.legend()

    @staticmethod
    def plot_decision_boundary(axes, Z: np.ndarray, X: np.ndarray, Y: np.ndarray):
        """
        Plot the decision boundary of the model.

        :param Z: Decision Function value for the whole grid.
        :param X: Samples.
        :param Y: Labels.
        """

        # Initialize the hyperparameters
        h = 0.01

        # Set min and max values and give it some padding
        x_min, x_max = X[0, :].min() - 1, X[0, :].max() + 1
        y_min, y_max = X[1, :].min() - 1, X[1, :].max() + 1

        # Generate a grid of points with distance h between them
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

        # Plot the decision boundary and training samples
        axes.set_ylabel("Feature #1")
        axes.set_xlabel("Feature #0")
        axes.set_title("Decision Boundary of the Model")

        axes.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
        train_scatter = axes.scatter(X[0, :], X[1, :], c=Y, cmap=plt.cm.Spectral)

        axes_legend = axes.legend(*train_scatter.legend_elements(), title="Classes")
        axes.add_artist(axes_legend)

    @staticmethod
    def plot_costs_and_decision_boundary(iterations: list, costs: list, learning_rate: float,
                                         Z: np.ndarray, X: np.ndarray, Y: np.ndarray):
        """
        Plot the cost of the network during training as well as the decision boundary of the model.

        :param iterations: Training iterations.
        :param costs: Costs of the network for each training iteration.
        :param learning_rate: Learning rate of the optimization algorithm.
        :param Z: Decision Function value for the whole grid.
        :param X: Samples.
        :param Y: Labels.
        """

        # Create both plots
        fig, (ax_costs, ax_decision) = plt.subplots(ncols=2, figsize=(16, 6))

        # Plot the costs
        Visualization.plot_the_cost(ax_costs, iterations, costs, learning_rate)

        # Plot the decision boundary
        Visualization.plot_decision_boundary(ax_decision, Z, X, Y)

<a id='2'></a>
## 2 - Gradient Descent \[6 points\] ##

A simple optimization method in machine learning is gradient descent (GD). As we already know, gradient descent goes "downhill" on a cost function $J$. At each step of the training, we update our parameters following a certain direction to try to get to the lowest possible point:

<div>
<img src="../img/optimization/gradient_descent_01.png" width="700"/>
</div>


When we imagine the cost function not as a a mathematical function but as a hilly landscape instead, we can visualize the gradient descent process as finding the lowest point in this hilly landscape as this:

<div>
<img src="../img/optimization/gradient_descent_02.png" width="700"/>
</div>

Created with and licensed under [Adobe Express](https://www.adobe.com/de/express/)

<a id='2-1'></a>
## 2.1 - Gradient Descent Update Rule \[1 point\] ##

**Remember:**
The gradient descent rule for $l = 1, ..., L$ is computed like:
$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]}$$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]}$$

<b><span style="color:teal">TODO:</span> <b>
<ol>
<li><span style="color:teal">Implement the method <span style="color:#DC143C"><em>gradient_descent</em></span> which runs one step of the gradient descent algorithm to update the model's parameters.</span></li>
</ol>

In [ ]:
def gradient_descent(parameters: dict, grads: dict, learning_rate: float):
    """
    Update parameters using one step of gradient descent.

    :param parameters: Dictionary containing the model parameters.
    :param grads: Dictionary containing the gradients of the cost function with respect to the model parameters.
    :param learning_rate: Learning rate for the gradient descent step.

    :return: Dictionary containing the updated model parameters
    """

    #############################################################################
    #                            START OF YOUR CODE                             #
    # TODO:                                                                     #
    #    1) Iterate through all layers.                                         #
    #    2) Update the weights of each layer.                                   #
    #    3) Update the bias of each layer.                                      #
    #############################################################################


    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################
    return parameters

In [ ]:
# Test your code

!python ./tests/test_optimization.py --test_case TestGradientDescent

In Deep Learning we differentiate between 3 types of gradient descent, namely "Stochastic Gradient Descent (SGD)", "Mini-batch Gradient Descent", and "Batch Gradient Descent". The following table illustrates their differences:


<div>
<img src="../img/optimization/gradient_descent_comparison_chart.png" width="1900"/>
</div>

The update rule that you have just implemented does not change for the three types. What changes is the number of training samples you would be computing gradients at a time.

**Remark:**
- "+" denotes a minimum of the cost. SGD leads to many oscillations to reach convergence, but each step is a lot faster to compute for SGD than it is for GD, as it uses only one training example (vs. the whole batch for GD).
- In practice, you'll often get faster results if you don't use the entire training set, or just one training sample, to perform each update.
- Mini-batch gradient descent uses an intermediate number of samples for each step. With mini-batch gradient descent, you loop over the mini-batches instead of looping over individual training examples.
- Using mini-batches in your optimization algorithm often leads to faster optimization.

<a id='2-2'></a>
## 2.2 - Stochastic Gradient Descent (SGD) ##

As you can see, Stochastic Gradient Descent (SGD) is equivalent to mini-batch gradient descent, where each mini-batch has just 1 sample. This means that we use only 1 training sample before updating the gradients. When the training set is large, SGD can be faster than batch gradient descent. But the parameters will "oscillate" toward the minimum rather than converge smoothly.

The code example below shows how you would train a network using SGD as the optimization method:

```python
X = data_input
Y = labels
m = X.shape[1]  # Number of training examples
parameters = initialize_parameters(layers_dims)
for i in range(0, num_iterations):
    cost_total = 0
    for j in range(0, m):
        # Forward propagation
        a, caches = forward_propagation(X[:,j], parameters)
        # Compute cost
        cost_total += compute_cost(a, Y[:,j])  # Cost for one training example
        # Backward propagation
        grads = backward_propagation(a, caches, parameters)
        # Update parameters
        parameters = update_parameters(parameters, grads)
    # Compute average cost
    cost_avg = cost_total / m
```

**Note** that implementing SGD requires 3 for-loops in total:
1. Over the number of iterations
2. Over the $m$ training samples
3. Over the layers (to update all parameters, from $(W^{[1]},b^{[1]})$ to $(W^{[L]},b^{[L]})$)

You don't have to code anything here.  This is just for your information.

<a id='2-3'></a>
## 2.3 - Batch Gradient Descent ##

For comparisopn to SGD, the code example below shows how you would train a network using batch gradient descent as the optimization method:

```python
X = data_input
Y = labels
m = X.shape[1]  # Number of training examples
parameters = initialize_parameters(layers_dims)
for i in range(0, num_iterations):
    # Forward propagation
    a, caches = forward_propagation(X, parameters)
    # Compute cost
    cost_total = compute_cost(a, Y)  # Cost for m training examples
    # Backward propagation
    grads = backward_propagation(a, caches, parameters)
    # Update parameters
    parameters = update_parameters(parameters, grads)
    # Compute average cost
    cost_avg = cost_total / m
```

Again you don't have to code anything here. This is just for your information.

<a id='2-4'></a>
## 2.4 - Mini-Batch Gradient Descent \[5 points\] ##

Mini-batch gradient descent seems promising, so let's try out! First, you will build some mini-batches from the training set (X, Y).

To do that, there are two steps involved:
- **Shuffle**: Create a shuffled version of the training set (X, Y) as shown below. Each column of X and Y represents a training example. Note that the random shuffling is done synchronously between X and Y. Such that after the shuffling the $i^{th}$ column of X is the example corresponding to the $i^{th}$ label in Y. The shuffling step ensures that examples will be split randomly into different mini-batches.

<div>
<img src="../img/optimization/mini_batch_gd_01.png" width="700"/>
</div>

- **Partition**: Partition the shuffled (X, Y) into mini-batches of size `mini_batch_size` (here 64). Note that the number of training examples is not always divisible by `mini_batch_size`. The last mini batch might be smaller, but you don't need to worry about this. When the final mini-batch is smaller than the full `mini_batch_size`, it will look like this:

<div>
<img src="../img/optimization/mini_batch_gd_02.png" width="700"/>
</div>


<b><span style="color:teal">TODO:</span> <b>
<ol>
<li><span style="color:teal">Implement the method <span style="color:#DC143C"><em>shuffle</em></span> which shuffles the input samples (X) and labels (Y).</span></li>
<li><span style="color:teal">Implement the method <span style="color:#DC143C"><em>random_mini_batches</em></span> which creates a list of random minibatches from the input (X, Y).</span></li>
</ol>

<b><span style="color:#B8860B">Hints:</span> <b>
<dl>
<dd><span style="color:#B8860B">1. Note that the last mini-batch might end up being smaller than the rest depending on the total number of samples provided. This has to be taken into consideration.</span></dd>
</dl>

In [ ]:
def shuffle(X: np.ndarray, Y: np.ndarray, seed: int):
    """
    Shuffle the input samples and labels randomly.

    :param X: Input data of shape (input size, number of samples).
    :param Y: Label vector of shape (1, number of samples).
    :param seed: Seed for the random operation(s).

    :return: Shuffled input samples, shuffled labels
    """

    shuffled_X, shuffled_Y = None, None

    #############################################################################
    #                            START OF YOUR CODE                             #
    # TODO:                                                                     #
    #    1) Create a "random" permutation by using np.random.permutation()      #
    #       - Remember to use the seed such that your results match ours.       #
    #    2) Shuffle X using the created permutation.                            #
    #    3) Shuffle Y using the created permutation.                            #
    #    4) Reshape Y to obtain its original shape.                             #
    #############################################################################


    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################

    return shuffled_X, shuffled_Y


def random_mini_batches(X: np.ndarray, Y: np.ndarray, mini_batch_size=64, seed=0):
    """
    Creates a list of random minibatches from (X, Y).

    :param X: Input data of shape (input size, number of samples).
    :param Y: Label vector of shape (1, number of samples).
    :param mini_batch_size: Size of the mini-batches.
    :param seed: Seed for the random operation(s).

    :return: List of synchronous (mini_batch_X, mini_batch_Y)
    """

    mini_batches = []

    #############################################################################
    #                            START OF YOUR CODE                             #
    # TODO:                                                                     #
    #    1) Use the upper shuffle-method to shuffle your input data and labels. #
    #                                                                           #
    #    2) For all "full" mini-batches:                                        #
    #       2.1) Extract the samples for the current mini-batch from the        #
    #            shuffled input data.                                           #
    #       2.2) Extract the labels for the current mini-batch from the         #
    #            shuffled labels.                                               #
    #       2.3) Create the current minibatch as (mini_batch_X, mini_batch_Y).  #
    #       2.4) Append the current mini-batch to the list of mini-batches.     #
    #                                                                           #
    #    3) Handle the case when the last mini-batch might end up being smaller #
    #       (e.g. size < 64) than the rest and append it to the list of         #
    #       mini-batches.                                                       #
    #############################################################################


    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################

    return mini_batches

In [ ]:
# Test your code

!python ./tests/test_optimization.py --test_case TestShuffling
!python ./tests/test_optimization.py --test_case TestMiniBatches

**What you should remember**:
- Shuffling and Partitioning are the two steps required to build mini-batches
- Powers of two are often chosen to be the mini-batch size, e.g., 16, 32, 64, 128.

<a id='3'></a>
## 3 - Momentum \[6 points\] ##

Because mini-batch gradient descent makes a parameter update after seeing just a subset of examples, the direction of the update has some variance, and so the path taken by mini-batch gradient descent will "oscillate" toward convergence. Using momentum can reduce these oscillations.

Momentum takes into account the past gradients to smooth out the update. The 'direction' of the previous gradients is stored in the variable $v$. Formally, this will be the exponentially weighted average of the gradient on previous steps. You can also think of $v$ as the "velocity" of a ball rolling downhill, building up speed (and momentum) according to the direction of the gradient/slope of the hill.

<div>
<img src="../img/optimization/momentum_01.png" width="700"/>
</div>

In the figure above, the red arrows show the direction taken by one step of mini-batch gradient descent with momentum. The blue points show the direction of the gradient (with respect to the current mini-batch) on each step. Rather than just following the gradient, the gradient is allowed to influence $v$ and then take a step in the direction of $v$.

<a id='3-1'></a>
## 3.1 - Initialize the Velocity $v$ \[2 points\] ##

Let's initialize the velocity $(v)$. The velocity is a python dictionary that needs to be initialized with arrays of zeros. Its keys are the same as those in the `grads` dictionary, that is:
for $l =1,...,L$:
```python
v["dW" + str(l)] = ... #(numpy array of zeros with the same shape as parameters["W" + str(l)])
v["db" + str(l)] = ... #(numpy array of zeros with the same shape as parameters["b" + str(l)])
```

<b><span style="color:teal">TODO:</span> <b>
<ol>
<li><span style="color:teal">Implement the method <span style="color:#DC143C"><em>initialize_velocity</em></span> which initializes the velocity as a python dictionary.</span></li>
</ol>

In [ ]:
def initialize_velocity(params: dict):
    """
    Initializes the velocity v.

    :param params: python dictionary containing your model parameters.
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl

    :return: velocity which is a python dictionary containing the current velocity.
                v['dW' + str(l)] = velocity of dWl
                v['db' + str(l)] = velocity of dbl
    """

    v = {}

    #############################################################################
    #                            START OF YOUR CODE                             #
    # TODO:                                                                     #
    #    1) Iterate through all layers.                                         #
    #    2) Initialize the velocity in the weight direction of each layer with  #
    #       zeros.                                                              #
    #    3) Initialize the velocity in the bias direction of each layer with    #
    #       zeros.                                                              #
    #############################################################################


    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################

    return v

In [ ]:
# Test your code

!python ./tests/test_optimization.py --test_case TestInitializingVelocity

<a id='3-2'></a>
## 3.2 - Update the Model's Parameters with Momentum \[4 points\] ##

Now, implement the parameters update with momentum. The momentum update rule is, for $l = 1, ..., L$:

\begin{align}
v_{dW^{[l]}} &= \beta v_{dW^{[l]}} + (1 - \beta) dW^{[l]} \\
W^{[l]} &= W^{[l]} - \alpha v_{dW^{[l]}}
\end{align}

\begin{align}
v_{db^{[l]}} &= \beta v_{db^{[l]}} + (1 - \beta) db^{[l]} \\
b^{[l]} &= b^{[l]} - \alpha v_{db^{[l]}}
\end{align}

where L is the number of layers, $\beta$ is the momentum and $\alpha$ is the learning rate.

<b><span style="color:teal">TODO:</span> <b>
<ol>
<li><span style="color:teal">Implement the method <span style="color:#DC143C"><em>update_parameters_with_momentum</em></span> which updates the model parameters using Momentum.</span></li>
</ol>

In [ ]:
def update_parameters_with_momentum(params: dict, grads: dict, v: dict, beta: float, learning_rate: float):
    """
    Update model parameters using Momentum.

    :param params: python dictionary containing your parameters
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    :param grads: python dictionary containing your gradients for each parameters:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    :param v: python dictionary containing the current velocity:
                    v['dW' + str(l)] = velocity of dWl
                    v['db' + str(l)] = velocity of dbl
    :param beta: The momentum.
    :param learning_rate: The learning rate.

    :return: updated model parameters, updated velocities
    """

    updated_params, updated_v = params, v

    #############################################################################
    #                            START OF YOUR CODE                             #
    # TODO:                                                                     #
    #    1) Iterate through all layers.                                         #
    #    2) Update the velocities using the equations above.                    #
    #    3) Update the model parameters using the equations above.              #
    #############################################################################


    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################

    return updated_params, updated_v

In [ ]:
# Test your code

!python ./tests/test_optimization.py --test_case TestMomentum

**Note that**:
- The velocity is initialized with zeros. So the algorithm will take a few iterations to "build up" velocity and start to take bigger steps.
- If $\beta = 0$, then this just becomes standard gradient descent without momentum.

**How do you choose $\beta$?**

- The larger the momentum $\beta$ is, the smoother the update, because it takes the past gradients into account more. But if $\beta$ is too big, it could also smooth out the updates too much.
- Common values for $\beta$ range from 0.8 to 0.999. If you don't feel inclined to tune this, $\beta = 0.9$ is often a reasonable default.
- Tuning the optimal $\beta$ for your model might require trying several values to see what works best in terms of reducing the value of the cost function $J$.

**What you should remember**:
- Momentum takes past gradients into account to smooth out the steps of gradient descent. It can be applied with batch gradient descent, mini-batch gradient descent or stochastic gradient descent.
- You have to tune a momentum hyperparameter $\beta$ and a learning rate $\alpha$.

<a id='4'></a>
## 4 - Adam \[6 points\] ##

[Adam](https://arxiv.org/pdf/1412.6980.pdf) is one of the most effective optimization algorithms for training neural networks. It combines ideas from [RMSProp and Momentum](https://blog.paperspace.com/intro-to-optimization-momentum-rmsprop-adam/).

**How does Adam work?**
1. It calculates an exponentially weighted average of past gradients, and stores it in variables $v$ (before bias correction) and $v^{corrected}$ (with bias correction).
2. It calculates an exponentially weighted average of the squares of the past gradients, and  stores it in variables $s$ (before bias correction) and $s^{corrected}$ (with bias correction).
3. It updates parameters in a direction based on combining information from "1" and "2".

The update rule is, for $l = 1, ..., L$:

\begin{align}
v_{dW^{[l]}} &= \beta_1 v_{dW^{[l]}} + (1 - \beta_1) \frac{\partial \mathcal{J} }{ \partial W^{[l]} } \\
v^{corrected}_{dW^{[l]}} &= \frac{v_{dW^{[l]}}}{1 - (\beta_1)^t} \\
s_{dW^{[l]}} &= \beta_2 s_{dW^{[l]}} + (1 - \beta_2) (\frac{\partial \mathcal{J} }{\partial W^{[l]} })^2 \\
s^{corrected}_{dW^{[l]}} &= \frac{s_{dW^{[l]}}}{1 - (\beta_2)^t} \\
W^{[l]} &= W^{[l]} - \alpha \frac{v^{corrected}_{dW^{[l]}}}{\sqrt{s^{corrected}_{dW^{[l]}}} + \varepsilon}
\end{align}

where:
- t counts the number of steps taken of Adam
- L is the number of layers
- $\beta_1$ and $\beta_2$ are hyperparameters that control the two exponentially weighted averages.
- $\alpha$ is the learning rate
- $\varepsilon$ is a very small number to avoid dividing by zero

Similar to gradient descent with momentum, we can visualize the process of mini-batch gradient descent with adam as the following:

<div>
<img src="../img/optimization/adam_01.png" width="700"/>
</div>

In the figure above, the red arrows show the direction taken by one step of mini-batch gradient descent with adam. The blue points show the direction of first momentum $v$ (exponentially weighted average of past gradients) with respect to the current mini-batch on each step. In contrast, The red points show the direction of the second momentum $s$ (exponentially weighted average of the squares of the past gradients) with respect to the current mini-batch on each step. Rather than just following the gradient, both momentum are allowed to the direction of the next gradient descent step.

<a id='4-1'></a>
## 4.1 - Initialize Adam \[2 points\] ##

Let's initialize the Adam variables $v$ (moving average of the first gradient) and $s$ (moving average of the squared gradient), which keep track of the past information.

<b><span style="color:teal">TODO:</span> <b>
<ol>
<li><span style="color:teal">Implement the method <span style="color:#DC143C"><em>initialize_adam</em></span> which initializes the variables $v$ and $s$ for the Adam optimization algorithm with zeros.</span></li>
</ol>

In [ ]:
def initialize_adam(params) :
    """
    Initializes v and s as two python dictionaries with:
        - keys: "dW1", "db1", ..., "dWL", "dbL"
        - values: numpy arrays of zeros of the same shape as the corresponding gradients/parameters.

    :param params: python dictionary containing your parameters
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl

    :return:    v -- python dictionary that will contain the exponentially weighted average of the gradient. Initialized with zeros.
                    v["dW" + str(l)] = ...
                    v["db" + str(l)] = ...
                s -- python dictionary that will contain the exponentially weighted average of the squared gradient. Initialized with zeros.
                    s["dW" + str(l)] = ...
                    s["db" + str(l)] = ...
    """

    v, s = {}, {}

    #############################################################################
    #                            START OF YOUR CODE                             #
    # TODO:                                                                     #
    #    1) Iterate through all layers.                                         #
    #    2) Initialize the entries of v for each layer as zeros with the        #
    #       respective shape.                                                   #
    #    3) Initialize the entries of s for each layer as zeros with the        #
    #       respective shape.                                                   #
    #############################################################################


    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################

    return v, s

In [ ]:
# Test your code

!python ./tests/test_optimization.py --test_case TestInitializingAdam

<a id='4-2'></a>
## 4.2 - Update Parameters using Adam \[4 points\] ##

Now, implement the parameters update with Adam. Recall the general update rule is, for $l = 1, ..., L$:

\begin{align}
v_{dW^{[l]}} &= \beta_1 v_{dW^{[l]}} + (1 - \beta_1) \frac{\partial \mathcal{J} }{ \partial W^{[l]} } \\
v_{db^{[l]}} &= \beta_1 v_{db^{[l]}} + (1 - \beta_1) \frac{\partial \mathcal{J} }{ \partial b^{[l]} } \\
v^{corrected}_{dW^{[l]}} &= \frac{v_{dW^{[l]}}}{1 - (\beta_1)^t} \\
v^{corrected}_{db^{[l]}} &= \frac{v_{db^{[l]}}}{1 - (\beta_1)^t} \\
s_{dW^{[l]}} &= \beta_2 s_{dW^{[l]}} + (1 - \beta_2) (\frac{\partial \mathcal{J} }{\partial W^{[l]} })^2 \\
s_{db^{[l]}} &= \beta_2 s_{db^{[l]}} + (1 - \beta_2) (\frac{\partial \mathcal{J} }{\partial b^{[l]} })^2 \\
s^{corrected}_{dW^{[l]}} &= \frac{s_{dW^{[l]}}}{1 - (\beta_2)^t} \\
s^{corrected}_{db^{[l]}} &= \frac{s_{db^{[l]}}}{1 - (\beta_2)^t} \\
W^{[l]} &= W^{[l]} - \alpha \frac{v^{corrected}_{dW^{[l]}}}{\sqrt{s^{corrected}_{dW^{[l]}}} + \varepsilon} \\
b^{[l]} &= b^{[l]} - \alpha \frac{v^{corrected}_{db^{[l]}}}{\sqrt{s^{corrected}_{db^{[l]}}} + \varepsilon} \\
\end{align}

<b><span style="color:teal">TODO:</span> <b>
<ol>
<li><span style="color:teal">Implement the method <span style="color:#DC143C"><em>update_parameters_with_adam</em></span> which updates the model parameters using Adam and stores the updated first and second moments for a next iteration.</span></li>
</ol>

In [ ]:
def update_parameters_with_adam(params, grads, v, s, t, learning_rate = 0.01, beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8):
    """
    Update model parameters using Adam.

    :param params: python dictionary containing your parameters
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    :param grads: python dictionary containing your gradients for each parameters:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    :param v: Adam variable (moving average of the first gradient).
    :param s: Adam variable (moving average of the squared gradient).
    :param t: Adam variable (counts the number of taken steps).
    :param beta1: Exponential decay hyperparameter for the first moment estimates.
    :param beta2: Exponential decay hyperparameter for the second moment estimates.
    :param epsilon: Hyperparameter preventing division by zero in Adam updates.
    :param learning_rate: The learning rate.

    :return: updated model parameters, updated v, updated s, bias corrected v, bias corrected s
    """

    v_corrected, s_corrected = {}, {}

    #############################################################################
    #                            START OF YOUR CODE                             #
    # TODO:                                                                     #
    #    1) Iterate through all layers.                                         #
    #                                                                           #
    #    2) Compute the updated moving average of the gradients (v) for         #
    #       the weights (v_dW) and the bias (v_db).                             #
    #                                                                           #
    #    3) Compute the bias-corrected first moment estimate (v_corrected) for  #
    #       the weights (v_corrected_dW) and the bias (v_corrected_db).         #
    #                                                                           #
    #    4) Compute the updated moving average of the squared gradients (s)     #
    #       for the weights (s_dW) and the bias (s_db).                         #
    #                                                                           #
    #    5) Compute the bias-corrected second raw moment estimate (s_corrected) #
    #       for the weights (s_corrected_dW) and the bias (s_corrected_db).     #
    #                                                                           #
    #    6) Update the model parameters using the previously updated variables. #
    #############################################################################


    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################
    return params, v, s, v_corrected, s_corrected

In [ ]:
# Test your code

!python ./tests/test_optimization.py --test_case TestAdam

<a name='5'></a>
## 5 - Model with different Optimization algorithms

You now have three working optimization algorithms (mini-batch gradient descent, Momentum, Adam). Let's implement a model with each of these optimizers and observe the difference.
Below, you'll use the following "moons" dataset to test the different optimization methods. (The dataset is named "moons" because the data from each of the two classes looks a bit like a crescent-shaped moon.)

In [ ]:
def load_dataset():
    """
    Load the sample "moons" dataset of sklearn.

    :return: samples, labels
    """
    np.random.seed(3)

    train_X, train_Y = sklearn.datasets.make_moons(n_samples=300, noise=.2) #300 #0.2

    # Visualize the data
    plt.scatter(train_X[:, 0], train_X[:, 1], c=train_Y, s=40, cmap=plt.cm.Spectral)

    train_X = train_X.T
    train_Y = train_Y.reshape((1, train_Y.shape[0]))

    return train_X, train_Y

# Load the dataset
train_X, train_Y = load_dataset()

A 3-layer neural network has already been implemented for you that was trained with:
- Mini-batch **Gradient Descent**
- Mini-batch **Momentum**
- Mini-batch **Adam**

<a name='5-1'></a>
### 5.1 - Mini-Batch Gradient Descent

Run the following code to see how the model does with mini-batch gradient descent.

In [ ]:
# Extract the data
with open("../data/optimization.pkl", "rb") as f:
    data = pickle.load(f)

learning_rate = data["gradient descent"]["learning_rate"]
iterations = data["gradient descent"]["iterations"]
costs = data["gradient descent"]["costs"]
decision = data["gradient descent"]["decision"]

# Plot the cost over the first 5000 iterations as well as the decision boundary of our model
Visualization.plot_costs_and_decision_boundary(iterations, costs, learning_rate, decision, train_X, train_Y)

# Print the accuracy of the model
accuracy = data["gradient descent"]["accuracy"]
print ("Accuracy of the model: ", accuracy)

<a name='5-2'></a>
### 5.2 - Mini-Batch Gradient Descent with Momentum

Next, run the following code to see how the model does with momentum. Because this example is relatively simple, the gains from using momemtum are small - but for more complex problems you might see bigger gains.

In [ ]:
# Extract the data
with open("../data/optimization.pkl", "rb") as f:
    data = pickle.load(f)

learning_rate = data["momentum"]["learning_rate"]
iterations = data["momentum"]["iterations"]
costs = data["momentum"]["costs"]
decision = data["momentum"]["decision"]

# Plot the cost over the first 5000 iterations as well as the decision boundary of our model
Visualization.plot_costs_and_decision_boundary(iterations, costs, learning_rate, decision, train_X, train_Y)

# Print the accuracy of the model
accuracy = data["momentum"]["accuracy"]
print ("Accuracy of the model: ", accuracy)

<a name='5-3'></a>
### 5.3 - Mini-Batch with Adam

Finally, run the following code to see how the model does with Adam.

In [ ]:
# Extract the data
with open("../data/optimization.pkl", "rb") as f:
    data = pickle.load(f)

learning_rate = data["adam"]["learning_rate"]
iterations = data["adam"]["iterations"]
costs = data["adam"]["costs"]
decision = data["adam"]["decision"]

# Plot the cost over the first 5000 iterations as well as the decision boundary of our model
Visualization.plot_costs_and_decision_boundary(iterations, costs, learning_rate, decision, train_X, train_Y)

# Print the accuracy of the model
accuracy = data["adam"]["accuracy"]
print ("Accuracy of the model: ", accuracy)

<a name='5-4'></a>
### 5.4 - Summary

<table>
    <tr>
        <td>
        <b>optimization method</b>
        </td>
        <td>
        <b>accuracy</b>
        </td>
        <td>
        <b>cost shape</b>
        </td>
    </tr>
        <td>
        Gradient descent
        </td>
        <td>
        >71%
        </td>
        <td>
        smooth
        </td>
    <tr>
        <td>
        Momentum
        </td>
        <td>
        >71%
        </td>
        <td>
        smooth
        </td>
    </tr>
    <tr>
        <td>
        Adam
        </td>
        <td>
        >94%
        </td>
        <td>
        smoother
        </td>
    </tr>
</table>

Momentum usually helps, but given the small learning rate and the simplistic dataset, its impact is almost negligible.

On the other hand, Adam clearly outperforms mini-batch gradient descent and Momentum. If you run the model for more epochs on this simple dataset, all three methods will lead to very good results. However, you've seen that Adam converges a lot faster.

Some advantages of Adam include:

- Relatively low memory requirements (though higher than gradient descent and gradient descent with momentum)
- Usually works well even with little tuning of hyperparameters (except $\alpha$)

<a name='6'></a>
## 6 - Learning Rate Decay and Scheduling \[2 points\]

Lastly, the learning rate is another hyperparameter that can help you speed up learning.

During the first part of training, your model can get away with taking large steps, but over time, using a fixed value for the learning rate alpha can cause your model to get stuck in a wide oscillation that never quite converges. But if you were to slowly reduce your learning rate alpha over time, you could then take smaller, slower steps that bring you closer to the minimum. This is the idea behind learning rate decay.

Learning rate decay can be achieved by using either adaptive methods or pre-defined learning rate schedules.

Now, you'll apply scheduled learning rate decay to a 3-layer neural network in three different optimizer modes and see how each one differs, as well as the effect of scheduling at different epochs.

This model is essentially the same as the one you used before, except that now learning rate decay was included during training. Hence, the model includes two new parameters, $\textit{decay}$ and $\textit{decay_rate}$.

<a name='6-1'></a>
### 6.1 - Decay on Every Iteration \[1 point\]

For this part, you'll try one of the pre-defined schedules for learning rate decay, called exponential learning rate decay. It takes this mathematical form:

$$\alpha = \frac{1}{1 + decayRate \times epochNumber} \alpha_{0}$$


<b><span style="color:teal">TODO:</span> <b>
<ol>
<li><span style="color:teal">Implement the method <span style="color:#DC143C"><em>update_lr</em></span> which updates the learning rate using exponential weight decay.</span></li>
</ol>

In [ ]:
def update_lr(learning_rate_0rig: float, epoch_num: int, decay_rate: float):
    """
    Update the learning rate using exponential learning rate decay.

    :param learning_rate_0rig: Original learning rate.
    :param epoch_num: Epoch number.
    :param decay_rate: Decay rate.

    :return: updated learning rate
    """

    updated_lr = None

    #############################################################################
    #                            START OF YOUR CODE                             #
    # TODO:                                                                     #
    #    1) Update the learning rate using the given equation.                  #
    #############################################################################


    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################

    return updated_lr

In [ ]:
# Test your code

!python ./tests/test_optimization.py --test_case TestExponentialWeightDecay

Now, let's apply the above implemented exponential learning rate decay to the training of our 3-layer neural network using gradient descent optimization from earlier. Run the cell below to see the results.

In [ ]:
# Extract the data
with open("../data/optimization.pkl", "rb") as f:
    data = pickle.load(f)

learning_rate = data["gd decay naive"]["learning_rate"]
iterations = data["gd decay naive"]["iterations"]
costs = data["gd decay naive"]["costs"]
decision = data["gd decay naive"]["decision"]

# Plot the cost over the first 5000 iterations as well as the decision boundary of our model
Visualization.plot_costs_and_decision_boundary(iterations, costs, learning_rate, decision, train_X, train_Y)

# Print the accuracy of the model
accuracy = data["gd decay naive"]["accuracy"]
print ("Accuracy of the model: ", accuracy)

Notice that if you set the decay to occur at every iteration, the learning rate goes to zero too quickly - even if you start with a higher learning rate.
<table>
    <tr>
        <td>
        <b>Epoch Number</b>
        </td>
        <td>
        <b>Learning Rate</b>
        </td>
        <td>
        <b>Cost</b>
        </td>
    </tr>
    <tr>
        <td>
        0
        </td>
        <td>
        0.100000
        </td>
        <td>
        0.701091
        </td>
    </tr>
    <tr>
        <td>
        1000
        </td>
        <td>
        0.000100
        </td>
        <td>
        0.661884
        </td>
    </tr>
    <tr>
        <td>
        2000
        </td>
        <td>
        0.000050
        </td>
        <td>
        0.658620
        </td>
    </tr>
    <tr>
        <td>
        3000
        </td>
        <td>
        0.000033
        </td>
        <td>
        0.656765
        </td>
    </tr>
    <tr>
        <td>
        4000
        </td>
        <td>
        0.000025
        </td>
        <td>
        0.655486
        </td>
    </tr>
    <tr>
        <td>
        5000
        </td>
        <td>
        0.000020
        </td>
        <td>
        0.654514
        </td>
    </tr>
</table>

When you're training for a few epoch this doesn't cause a lot of troubles, but when the number of epochs is large the optimization algorithm will stop updating. One common fix to this issue is to decay the learning rate every few steps. This is called fixed interval scheduling.

<a name='6-2'></a>
### 6.2 - Fixed Interval Scheduling \[1 point\]

You can help prevent the learning rate speeding to zero too quickly by scheduling the exponential learning rate decay at a fixed time interval, for example 1000. You can either number the intervals, or divide the epoch by the time interval, which is the size of window with the constant learning rate.

<div>
<img src="../img/optimization/learning_rate_01.png" width="700"/>
</div>

This process takes the following mathematical form:

$$\alpha = \frac{1}{1 + decayRate \times \lfloor\frac{epochNum}{timeInterval}\rfloor} \alpha_{0}$$

The fraction in the denominator uses the [floor](https://numpy.org/doc/stable/reference/generated/numpy.floor.html) operation.


<b><span style="color:teal">TODO:</span> <b>
<ol>
<li><span style="color:teal">Implement the method <span style="color:#DC143C"><em>schedule_lr_decay</em></span> which updates the learning rate using exponential weight decay with fixed interval scheduling.</span></li>
</ol>

In [ ]:
def schedule_lr_decay(learning_rate_0rig: float, epoch_num: int, decay_rate: float, time_interval=1000):
    """
    Update the learning rate using exponential learning rate decay with fixed interval scheduling.

    :param learning_rate_0rig: Original learning rate.
    :param epoch_num: Epoch number.
    :param decay_rate: Decay rate.
    :param time_interval: Number of epochs where you update the learning rate.

    :return: updated learning rate
    """

    updated_lr = None

    #############################################################################
    #                            START OF YOUR CODE                             #
    # TODO:                                                                     #
    #    1) Update the learning rate using the given equation.                  #
    #############################################################################


    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################

    return updated_lr

In [ ]:
# Test your code

!python ./tests/test_optimization.py --test_case TestExponentialWeightDecayScheduled

<a name='6-3'></a>
### 6.3 - Using Scheduled Learning Rate Decay for each Optimization Method

Below, you'll use the "moons" dataset again to test the different optimization methods.

<a name='6-3-1'></a>
#### 6.3.1 - Gradient Descent with Scheduled Learning Rate Decay

Run the following code to see how the model does, using gradient descent with scheduled weight decay.

In [ ]:
# Extract the data
with open("../data/optimization.pkl", "rb") as f:
    data = pickle.load(f)

learning_rate = data["gd decay scheduled"]["learning_rate"]
iterations = data["gd decay scheduled"]["iterations"]
costs = data["gd decay scheduled"]["costs"]
decision = data["gd decay scheduled"]["decision"]

# Plot the cost over the first 5000 iterations as well as the decision boundary of our model
Visualization.plot_costs_and_decision_boundary(iterations, costs, learning_rate, decision, train_X, train_Y)

# Print the accuracy of the model
accuracy = data["gd decay scheduled"]["accuracy"]
print ("Accuracy of the model: ", accuracy)

<a name='6-3-2'></a>
#### 6.3.2 - Gradient Descent with Momentum and Scheduled Learning Rate Decay

Run the following code to see how the model does, using gradient descent with momentum and scheduled weight decay.

In [ ]:
# Extract the data
with open("../data/optimization.pkl", "rb") as f:
    data = pickle.load(f)

learning_rate = data["momentum decay scheduled"]["learning_rate"]
iterations = data["momentum decay scheduled"]["iterations"]
costs = data["momentum decay scheduled"]["costs"]
decision = data["momentum decay scheduled"]["decision"]

# Plot the cost over the first 5000 iterations as well as the decision boundary of our model
Visualization.plot_costs_and_decision_boundary(iterations, costs, learning_rate, decision, train_X, train_Y)

# Print the accuracy of the model
accuracy = data["momentum decay scheduled"]["accuracy"]
print ("Accuracy of the model: ", accuracy)

<a name='6-3-3'></a>
#### 6.3.3 - Adam with Scheduled Learning Rate Decay

Run the following code to see how the model does, using Adam and scheduled weight decay.

In [ ]:
# Extract the data
with open("../data/optimization.pkl", "rb") as f:
    data = pickle.load(f)

learning_rate = data["adam decay scheduled"]["learning_rate"]
iterations = data["adam decay scheduled"]["iterations"]
costs = data["adam decay scheduled"]["costs"]
decision = data["adam decay scheduled"]["decision"]

# Plot the cost over the first 5000 iterations as well as the decision boundary of our model
Visualization.plot_costs_and_decision_boundary(iterations, costs, learning_rate, decision, train_X, train_Y)

# Print the accuracy of the model
accuracy = data["adam decay scheduled"]["accuracy"]
print ("Accuracy of the model: ", accuracy)

<a name='6-4'></a>
### 6.4 - Achieving similar performance with different methods

With Mini-batch GD or Mini-batch GD with Momentum, the accuracy is significantly lower than Adam, but when learning rate decay is added on top, either can achieve performance at a speed and accuracy score that's similar to Adam.

In the case of Adam, notice that the learning curve achieves a similar accuracy but faster.

<table>
    <tr>
        <td>
        <b>optimization method</b>
        </td>
        <td>
        <b>accuracy</b>
        </td>
    </tr>
        <td>
        Gradient descent
        </td>
        <td>
        >94.6%
        </td>
    <tr>
        <td>
        Momentum
        </td>
        <td>
        >95.6%
        </td>
    </tr>
    <tr>
        <td>
        Adam
        </td>
        <td>
        94%
        </td>
    </tr>
</table>

<a id='7'></a>
## 7 - End of Exercise ##

<div>
<img src="../img/memes/meme_congrats_03.png" width="700"/>
</div>

Created with and licensed under [Adobe Express](https://www.adobe.com/de/express/)